__author__ = 'Ricardo Pasquini'

# Counting tweets at the hexagon level. Complete procedure

Overview: This code counts georeferenced tweets at the hexagon level. Hexagons are uniquely identified in space. We use Uber's H3 library for this purpose.  
The code  also performs a home location prediction for each user. This allows later counting tweets by whether they were tweeted by residents, residents of neighbors hexagons, or non-residents.
The project was designed to be scalable to millions of tweets. The process is done with MongoDB.

By default the project generates the following collections on the Mongo database:

tweets: Collection containing tweets. This collection is modified by incorporating the corresponding hex id. \
users: Collection containing users. This collection is modified by incorporating the home location data.\
hexcounts: Collection containing counts of tweets.


In [1]:
import sys
sys.path.append("../") 
import databasepopulation
import communicationwmongo as commu
import home_location as home
import analysis as a
import pymongo

In [10]:
import importlib
importlib.reload(databasepopulation)
importlib.reload(home)
importlib.reload(a)

<module 'analysis' from '..\\analysis.py'>

# Counting tweets at the hexagon level. Complete procedure

This version of the code assumes that a different database will be used for each city.


If necessary check your current location with the following command:

In [3]:
pwd

'C:\\Users\\emman\\Documents\\git\\twitter_and_displacement\\notebooks'

# 0. Connect to Mongo and define a specific database

By default connect to Mongo running on a localhost. 
Requires manually changing the parameters if working on a cloud environment

I will use the name buenosaires as the db example. Recall each database for each city.


In [4]:
db=commu.connecttoLocaldb(database='Twitter_Data')

In [5]:
db.Twitter_Data

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Twitter_Data'), 'Twitter_Data')

# 1. Raw data to Mongo
This function populates the files containing tweets into Mongo.
Assumes files are named like: ba_2012.csv, etc.
Choose start and end years.


In [7]:
databasepopulation.populatetweets(db, path='/Users/emman/Box/Twitter data/Asia/Sydney/', cityprefix='sydney', yearstart=2012, yearend=2012)

Now populating year  2012
process completed


# 2. Adding Hexids to tweets 
This function includes hexagons ids to each tweet.  

In [11]:
databasepopulation.addhexjob(db)

KeyError: 'lat'

In [ ]:
# the following is a check and a fix because I think the previous process did not run completely, probably a bug.
# are there any other documents that were not processed?
len(list(db.tweets.find({ "hex" : { "$exists" : False } })))

In [ ]:
#Process any missing document
cursor=db.tweets.find({ "hex" : { "$exists" : False } })
df = pd.DataFrame(list(cursor))
requests = databasepopulation.add_hexs_and_prepare_bulk_request(df, dataformat='raw')
try:
    db.tweets.bulk_write(requests, ordered=False)
except BulkWriteError as bwe:
    print(bwe.details)

# 3. Database performance task: Add indexes to tweets

In [ ]:
databasepopulation.create_indexes(db)

# 4. Populate Users Collection

In [ ]:
databasepopulation.populate_users_collection(db)

# 5. Find Home Job for each user id in the database

In [ ]:
home.job_findhomeandpopulate_hex9(db)

In [ ]:
print('Users with home location identified', db.users.count_documents({'hex9': { '$exists': True} }))


# 6 Generate a Hex-level collection including all hexs in the database

By default the collection include as ids only the ids of hexagons at resolution 9
Note that many types of hexogons could coexist in a same collection because hex identifiers are unique. This is not implemented in the current version of the code, though.

In [ ]:
databasepopulation.populate_hexcounts_collection(db)


# 7 Count tweets in each hex by residents and non-residents

In [ ]:
import analysis as a

In [ ]:
a.countandpopulatejob(db)

# 8 Query to the DB to create the counts dataframe 

In [ ]:
df=a.hexcountsresults_to_df(db, save=False)
# if save=True then saves resulting dataframe a pickle in ./hexcountsdf.pkl

In [ ]:
df